# Transformer based models

### Importing the required libraries

In [ ]:
# ! pip install allennlp
# ! pip install allennlp-models
! pip install nltk==3.5

import io
import numpy as np
# from google_trans_new import google_translator 
from scipy.spatial.distance import cosine
import json 
import string
import nltk
nltk.download('stopwords')
from nltk.corpus import stopwords
stop_words = set(stopwords.words('english'))
from nltk import word_tokenize
from itertools import groupby 
from spacy.lang.hi import STOP_WORDS as STOP_WORDS_HI

from tqdm import tqdm
import pandas as pd

!pip install indic-nlp-library
import indicnlp
from indicnlp.tokenize import indic_tokenize 
!pip install -U torch
import torch
!pip install -U transformers
!pip install -U sentence_transformers
import transformers
from sentence_transformers import SentenceTransformer,util
import numpy as np
!pip install google_trans_new
from google_trans_new import google_translator
!pip install stanfordnlp
import stanfordnlp 
stanfordnlp.download('hi')
nlp = stanfordnlp.Pipeline(processors = "tokenize,pos",lang = 'hi')

In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
# model = SentenceTransformer('stsb-xlm-r-multilingual')
# model = SentenceTransformer('distiluse-base-multilingual-cased-v2')
# model = SentenceTransformer('distiluse-base-multilingual-cased-v2')
model = SentenceTransformer('paraphrase-xlm-r-multilingual-v1')
# model = SentenceTransformer('msmarco-distilbert-base-v2')
# model = SentenceTransformer('LaBSE')

In [ ]:
pos_dict = {
'CC': 'coordinating conjunction','CD': 'cardinal digit','DT': 'determiner',
'EX': 'existential there (like: \"there is\" ... think of it like \"there exists\")',
'FW': 'foreign word','IN':  'preposition/subordinating conjunction','JJ': 'adjective \'big\'',
'JJR': 'adjective, comparative \'bigger\'','JJS': 'adjective, superlative \'biggest\'',
'LS': 'list marker 1)','MD': 'modal could, will','NN': 'noun, singular \'desk\'',
'NNS': 'noun plural \'desks\'','NNP': 'proper noun, singular \'Harrison\'',
'NNPS': 'proper noun, plural \'Americans\'','PDT': 'predeterminer \'all the kids\'',
'POS': 'possessive ending parent\'s','PRP': 'personal pronoun I, he, she',
'PRP$': 'possessive pronoun my, his, hers','RB': 'adverb very, silently,',
'RBR': 'adverb, comparative better','RBS': 'adverb, superlative best',
'RP': 'particle give up','TO': 'to go \'to\' the store.','UH': 'interjection errrrrrrrm',
'VB': 'verb, base form take','VBD': 'verb, past tense took',
'VBG': 'verb, gerund/present participle taking','VBN': 'verb, past participle taken',
'VBP': 'verb, sing. present, non-3d take','VBZ': 'verb, 3rd person sing. present takes',
'WDT': 'wh-determiner which','WP': 'wh-pronoun who, what','WP$': 'possessive wh-pronoun whose',
'WRB': 'wh-abverb where, when','QF' : 'quantifier, bahut, thoda, kam (Hindi)','VM' : 'main verb',
'PSP' : 'postposition, common in indian langs','DEM' : 'demonstrative, common in indian langs'
}



### Loading data from json file

In [2]:
import json 
# Opening JSON file 
testA,testC,testP = open('/content/drive/MyDrive/test_data/testA.json',encoding = 'utf-8') ,open('/content/drive/MyDrive/test_data/testC2.json',encoding = 'utf-8'),open('/content/drive/MyDrive/test_data/testP.json',encoding = 'utf-8')
# testA,testC,testP = open('/content/drive/MyDrive/test_data/testA.json',encoding = 'utf-8') ,open('/content/drive/MyDrive/test_data/testC2.json',encoding = 'utf-8'),open('/content/drive/MyDrive/test_data/politicians.json',encoding = 'utf-8')
# testA,testC,testP = open('/content/drive/MyDrive/test_data/testA.json',encoding = 'utf-8') ,open('/content/drive/MyDrive/test_data/testC2.json',encoding = 'utf-8'),open('/content/drive/MyDrive/test_data/corrected_annotated_data/politicians_test(corrected).json',encoding = 'utf-8')
dataA,dataC,dataP = json.load(testA),json.load(testC),json.load(testP),
# returns JSON object as  
# a dictionary 
# data = json.load(f) 
# d = {}
# d = data
  
# Closing file 
# f.close()


In [ ]:
for qid in dataC:
  t =  dataC[qid]
  for key in t:
    if key == 'triples':
      triplelist = t[key]
      for l in triplelist:
        if len(l) == 1:
          triplelist.remove(l)

In [ ]:
### Getting a_c_p for getting all the triples
# Opening JSON file 
f = open('/content/drive/MyDrive/a_c_p.json',) 
data = json.load(f) 
acp = {}
acp = data
f.close() 

# Sentence to Triple Matching

### Obtaining sentence and triple embeddings
We obtain the sentence embeddings by taking each sentence, obtaining word embeddings for each word in the sentence, and averaging the word embeddings. For triple embeddings , we average the embeddings for each word in the triple.

In [ ]:
# Matching Triples with sentences
actors_test, cricketers_test, politicians_test = dataA, dataC, dataP
actors, cricketers, politicians = actors_test, cricketers_test, politicians_test

# Creating the dictionary for the test data where key = sentence and value = list of matching triples
actors_test_dict, cricketers_test_dict, politicians_test_dict = {}, {}, {}
for l in actors_test:
    for k, v in actors_test[l].items():
        if k == 'sentence':
            sentence = v
        if k == 'triples':
            triple_list = v
    t = [(e['subject'], e['predicate'], e['object']) for e in triple_list]
    actors_test_dict[sentence] = set(t)

for l in cricketers_test:
    for k, v in cricketers_test[l].items():
        if k == 'sentence':
            sentence = v
        if k == 'triples':
            triple_list = v
    t = [(e['subject'], e['predicate'], e['object']) for e in triple_list]
    cricketers_test_dict[sentence] = set(t)

for l in politicians_test:
    for k, v in politicians_test[l].items():
        if k == 'sentence':
            sentence = v
        if k == 'triples':
            triple_list = v
    t = [(e['subject'], e['predicate'], e['object']) for e in triple_list]
    politicians_test_dict[sentence] = set(t)
# for l in politicians_test:
#   entries = politicians_test[l]
#   subject = entries['personLabel']
#   triple_list = entries['triples']
  
    
len(actors_test_dict),len(cricketers_test_dict),len(politicians_test_dict)

(50, 100, 100)

Keeping only the gold standard Sentences in politcians

Cleaning the annotated triples --  We need to prune out certain triples in politicians -- like date of birth, data of death in predicate, British India and Dominion of India in objects.

In [ ]:
#@title
temp = {}
for sentence in politicians_test_dict:
  triplelist = politicians_test_dict[sentence]
  new_triple_list = set()
  for triple in triplelist:
    if triple[1] == 'date of birth' or triple[1] == 'date of death' or 'name' in triple[1] or triple[1] == 'sex or gender' or triple[2] == 'British India' or triple[2] == 'Dominion of India':
      continue
    else:
      new_triple_list.add(triple)
    temp[sentence] = new_triple_list
politicians_test_dict = temp

temp = {}
for sentence in actors_test_dict:
  triplelist = actors_test_dict[sentence]
  new_triple_list = set()
  for triple in triplelist:
    if triple[1] == 'date of birth' or triple[1] == 'date of death' or 'name' in triple[1] or triple[1] == 'sex or gender' or triple[2] == 'British India' or triple[2] == 'Dominion of India':
      continue
    else:
      new_triple_list.add(triple)
    temp[sentence] = new_triple_list
actors_test_dict = temp

temp = {}
for sentence in cricketers_test_dict:
  triplelist = cricketers_test_dict[sentence]
  new_triple_list = set()
  for triple in triplelist:
    if triple[1] == 'date of birth' or triple[1] == 'date of death' or triple[1] == 'sex or gender' or triple[2] == 'British India' or triple[2] == 'Dominion of India':
      continue
    else:
      new_triple_list.add(triple)
    temp[sentence] = new_triple_list
cricketers_test_dict = temp

Loading ACP.json for training

In [ ]:
f = open('/content/drive/MyDrive/test_data/politicians.json',encoding = 'utf-8')
acp_p = json.load(f)
f = open('/content/drive/MyDrive/test_data/actors.json',encoding = 'utf-8')
acp_a = json.load(f)
f = open('/content/drive/MyDrive/test_data/cricketers.json',encoding = 'utf-8')
acp_c = json.load(f)

In [ ]:
#@title
# #@title
# # We had got a_c_p.json. We keep only the relevant triples by filtering by entity id in test annotated data
# actors_trip, cricketers_trip, politician_trip = [], [], []
# actors_sent, cricketers_sent, politician_sent = [], [], []

# # Putting actors,cricketers and politicians from a_c_p
# # act, cric, pol = acp['a'], acp['c'], acp['p']      # Original acp.json

# # Now, we are using better, cleaned , pruned version of acp json, ( cleaned by Shivprasad)
# act = acp_a
# cric = acp_c
# pol = acp_p
# ############### Actors #######################

# # Iterating over test annotated data and keeping relevant triples only
# entity_tracking = []
# for e in actors_test:
#     eid = actors_test[e]['entity_id']
#     for ele in act:
#         if eid == ele and eid not in entity_tracking:
#             entity_tracking.append(eid)
#             # Getting triples for the matching entity id
#             triples = act[ele]['triples']
#             subject = act[ele]['personLabel']
#             triplist = []
#             for trip in triples:
#                 predicate = trip['propertyLabel']
#                 obj = trip['objectLabel']
#                 trip_tuple = (subject, predicate, obj)
#                 triplist.append(trip_tuple)
#             actors_trip.append(triplist)

# # Iterating over test annotated data and grouping annotated sentences together by entity id

# for eid in entity_tracking:
#     sentence_list = []
#     for e in actors_test:
#         if eid == actors_test[e]['entity_id']:
#             sentence = actors_test[e]['sentence']
#             sentence_list.append(sentence)
#     actors_sent.append(sentence_list)


# ############### Cricketers #######################

# # Iterating over test annotated data and keeping relevant triples only
# entity_tracking = []
# for e in cricketers_test:
#     eid = cricketers_test[e]['entity_id']
#     for ele in cric:
#         if eid == ele and eid not in entity_tracking:
#             entity_tracking.append(eid)
#             # Getting triples for the matching entity id
#             triples = cric[ele]['triples']
#             subject = cric[ele]['personLabel']
#             triplist = []
#             for trip in triples:
#                 predicate = trip['propertyLabel']
#                 obj = trip['objectLabel']
#                 trip_tuple = (subject, predicate, obj)
#                 triplist.append(trip_tuple)
#             cricketers_trip.append(triplist)

# # Iterating over test annotated data and grouping annotated sentences together by entity id

# for eid in entity_tracking:
#     sentence_list = []
#     for e in cricketers_test:
#         if eid == cricketers_test[e]['entity_id']:
#             sentence = cricketers_test[e]['sentence']
#             sentence_list.append(sentence)
#     cricketers_sent.append(sentence_list)


# ############### Politicians #######################

# # Iterating over test annotated data and keeping relevant triples only
# entity_tracking = []
# for e in politicians_test:
#     eid = politicians_test[e]['entity_id']
#     for ele in pol:
#         if eid == ele and eid not in entity_tracking:
#             entity_tracking.append(eid)
#             # Getting triples for the matching entity id
#             triples = pol[ele]['triples']
#             subject = pol[ele]['personLabel']
#             triplist = []
#             for trip in triples:
#                 predicate = trip['propertyLabel']
#                 obj = trip['objectLabel']
#                 trip_tuple = (subject, predicate, obj)
#                 triplist.append(trip_tuple)
#             politician_trip.append(triplist)

# # Iterating over test annotated data and grouping annotated sentences together by entity id

# for eid in entity_tracking:
#     sentence_list = []
#     for e in politicians_test:
#         if eid == politicians_test[e]['entity_id']:
#             sentence = politicians_test[e]['sentence']
#             sentence_list.append(sentence)
#     politician_sent.append(sentence_list)

In [ ]:
#@title
# We had got a_c_p.json. We keep only the relevant triples by filtering by entity id in test annotated data
actors_trip, cricketers_trip, politician_trip = [], [], []
actors_sent, cricketers_sent, politician_sent = [], [], []

# Putting actors,cricketers and politicians from a_c_p
# act, cric, pol = acp['a'], acp['c'], acp['p']      # Original acp.json

# Now, we are using better, cleaned , pruned version of acp json, ( cleaned by Shivprasad)
act = acp_a
cric = acp_c
pol = acp_p
############### Actors #######################

# Iterating over test annotated data and keeping relevant triples only
entity_tracking = []
for e in actors_test:
    eid = actors_test[e]['entity_id']
    for ele in act:
        if eid == ele and eid not in entity_tracking:
            entity_tracking.append(eid)
            # Getting triples for the matching entity id
            triples = act[ele]['triples']
            subject = act[ele]['personLabel']
            triplist = []
            for trip in triples:
                predicate = trip[1]
                obj = trip[2]
                trip_tuple = (subject, predicate, obj)
                triplist.append(trip_tuple)
            actors_trip.append(triplist)

# Iterating over test annotated data and grouping annotated sentences together by entity id
actors_train_sent = []    ##Group train sentences in list of lists
for eid in entity_tracking:
    sentence_list = []
    for e in actors_test:
        if eid == actors_test[e]['entity_id']:
            sentence = actors_test[e]['sentence']
            sentence_list.append(sentence)
            train_sentence = act[eid]['sentences']
    actors_sent.append(sentence_list)
    actors_train_sent.append(train_sentence)
######     Keyphrase  ##################
# 1. Make it so that sentences (in training data act) are grouped together. 
############### Cricketers #######################

# Iterating over test annotated data and keeping relevant triples only

entity_tracking = []
for e in cricketers_test:
    eid = cricketers_test[e]['entity_id']
    for ele in cric:
        if eid == ele and eid not in entity_tracking:
            entity_tracking.append(eid)
            # Getting triples for the matching entity id
            triples = cric[ele]['triples']
            subject = cric[ele]['personLabel']
            triplist = []
            for trip in triples:
                predicate = trip['propertyLabel']
                obj = trip['objectLabel']
                trip_tuple = (subject, predicate, obj)
                triplist.append(trip_tuple)
            cricketers_trip.append(triplist)

# Iterating over test annotated data and grouping annotated sentences together by entity id
cricketers_train_sent = []
for eid in entity_tracking:
    sentence_list = []
    for e in cricketers_test:
        if eid == cricketers_test[e]['entity_id']:
            sentence = cricketers_test[e]['sentence']
            sentence_list.append(sentence)
            train_sentence = acp['c'][eid]['sentences']
    cricketers_sent.append(sentence_list)
    cricketers_train_sent.append(train_sentence)

############### Politicians #######################

# Iterating over test annotated data and keeping relevant triples only
entity_tracking = []
for e in politicians_test:
    eid = politicians_test[e]['entity_id']
    for ele in pol:
        if eid == ele and eid not in entity_tracking:
            entity_tracking.append(eid)
            # Getting triples for the matching entity id
            triples = pol[ele]['triples']
            subject = pol[ele]['personLabel']
            triplist = []
            for trip in triples:
                predicate = trip['propertyLabel']
                obj = trip['objectLabel']
                trip_tuple = (subject, predicate, obj)
                triplist.append(trip_tuple)
            politician_trip.append(triplist)

# Iterating over test annotated data and grouping annotated sentences together by entity id
politicians_train_sent = []
for eid in entity_tracking:
    sentence_list = []
    for e in politicians_test:
        if eid == politicians_test[e]['entity_id']:
            sentence = politicians_test[e]['sentence']
            sentence_list.append(sentence)
            train_sentence = pol[eid]['sentences']
    politician_sent.append(sentence_list)
    politicians_train_sent.append(train_sentence)

In [ ]:
len(actors_train_sent),len(cricketers_train_sent),len(politicians_train_sent)

(50, 100, 99)

In [ ]:
len(actors_sent), len(cricketers_sent),len(politician_sent)

(50, 100, 99)

In [ ]:
len(actors_trip), len(cricketers_trip),len(politician_trip)

(50, 100, 99)

# Keyphrase Ranking

In [ ]:
# We have obtained the ordered train list of lists (in same order as annotated sentences)
# Now, maintaining the same order , we create a train list of list of keyphrases
from nltk import RegexpParser

def extract_pos(doc):
    pos = [(wrd.text,wrd.pos) for sent in doc.sentences for wrd in sent.words]
    return pos
def get_kp(sent):
  hindi = nlp(sent)
  extract_pos(hindi)

  pos_tags = extract_pos(hindi)
  patterns = "NP: {<JJ>*<NN|NNS|NNC|NNCS|NNP|NNPS|NNPC|NNPCS>+}"
  PChunker = RegexpParser(patterns)
  output = PChunker.parse(pos_tags)
  keyphrases = []
  for subtree in output.subtrees(filter=lambda t: t.label() == 'NP'):
      keyphrases.append((' '.join([x[0] for x in subtree])))
  return keyphrases

def rank_keyphrases(total_keyphrases,individual_keyphrases):
  article_keyphrases = ' '.join(total_keyphrases)
  article_keyphrases_embedding = model.encode(article_keyphrases,convert_to_tensor= True)
  individual_embeddings_of_keyphrases = model.encode(individual_keyphrases,convert_to_tensor= True)
  keyphrase_scores = []
  for keyphrase,embedding in zip(total_keyphrases,individual_embeddings_of_keyphrases):
    cosine_scores = util.pytorch_cos_sim(article_keyphrases_embedding, embedding)
    keyphrase_scores.append([cosine_scores.item(),keyphrase])
  keyphrase_scores.sort(reverse=True)
  return keyphrase_scores
  # for score,kp in keyphrase_scores:
  #   print(kp," : ",score)

### Getting Total Keyphrases (from train sentences) ###
actors_total_keyphrases,cricketers_total_keyphrases,politicians_total_keyphrases = [],[],[]
for sentence_list in actors_train_sent:
  total_keyphrases = []
  for sentence in sentence_list:
    keyphrase = get_kp(sentence)
    total_keyphrases.extend(keyphrase)
  actors_total_keyphrases.append(total_keyphrases)

for sentence_list in cricketers_train_sent:
  total_keyphrases = []
  for sentence in sentence_list:
    keyphrase = get_kp(sentence)
    total_keyphrases.extend(keyphrase)
  cricketers_total_keyphrases.append(total_keyphrases)

for sentence_list in politicians_train_sent:
  total_keyphrases = []
  for sentence in sentence_list:
    keyphrase = get_kp(sentence)
    total_keyphrases.extend(keyphrase)
  politicians_total_keyphrases.append(total_keyphrases)

### Getting Individual Keyphrases (from test sentences) ###
actors_individual_keyphrases,cricketers_individual_keyphrases,politicians_individual_keyphrases = [],[],[]
for sentence_list in actors_sent:
  total_sents = ""
  for sentence in sentence_list:
    total_sents = total_sents + sentence
  keyphrases = get_kp(total_sents)
  actors_individual_keyphrases.append(keyphrases)

for sentence_list in cricketers_sent:
  total_sents = ""
  for sentence in sentence_list:
    total_sents = total_sents + sentence
  keyphrases = get_kp(total_sents)
  cricketers_individual_keyphrases.append(keyphrases)

for sentence_list in politician_sent:
  total_sents = ""
  for sentence in sentence_list:
    total_sents = total_sents + sentence
  keyphrases = get_kp(total_sents)
  politicians_individual_keyphrases.append(keyphrases)

In [ ]:
len(actors_total_keyphrases),len(cricketers_total_keyphrases),len(politicians_total_keyphrases)

(50, 100, 99)

In [ ]:
len(actors_individual_keyphrases),len(cricketers_individual_keyphrases),len(politicians_individual_keyphrases)

(50, 100, 99)

In [ ]:
actors_keyphrase_scores, cricketers_keyphrase_scores, politicians_keyphrase_scores = [],[],[]
for total_keyphrases,individual_keyphrases in zip(actors_total_keyphrases,actors_individual_keyphrases):
  keyphrase_scores = rank_keyphrases(total_keyphrases,individual_keyphrases)
  actors_keyphrase_scores.append(keyphrase_scores)

for total_keyphrases,individual_keyphrases in zip(cricketers_total_keyphrases,cricketers_individual_keyphrases):
  keyphrase_scores = rank_keyphrases(total_keyphrases,individual_keyphrases)
  cricketers_keyphrase_scores.append(keyphrase_scores)

for total_keyphrases,individual_keyphrases in zip(politicians_total_keyphrases,politicians_individual_keyphrases):
  keyphrase_scores = rank_keyphrases(total_keyphrases,individual_keyphrases)
  politicians_keyphrase_scores.append(keyphrase_scores)

In [ ]:
len(actors_keyphrase_scores), len(cricketers_keyphrase_scores), len(politicians_keyphrase_scores)

(50, 100, 99)

# Alignment

Finding Top K matches

In [ ]:
#@title
from nltk.util import ngrams
from nltk import RegexpParser

def preprocess(triple_list):
  new_triple_list = []
  for s,p,o in triple_list:
    if p == 'date of birth' or p == 'date of death' or 'name' in p or p == 'sex or gender' or o == 'British India' or o == 'Dominion of India':
      continue
    else:
      new_triple_list.append((s,p,o))
  return new_triple_list

def matches_kp(sentences,triples,keyphrase_scores):       # Matches based on keyphrase
    matches_dict = {}                 # Dictionary to store the sentences with the matching triples. key = sent, value = triple
    for sent in sentences:
      sentence2 =  sent
      triple_list = triples
      triple_list = preprocess(triple_list)
      matches = []
      # keyphrases = ['ईरानी ऑस्ट्रियाई राजनीतिक कार्यकर्ता है']
      for score,kp in keyphrase_scores:
        if score <0.2:
          continue
        words = [w for w in indic_tokenize.trivial_tokenize(kp, lang = 'hi') if w not in STOP_WORDS_HI]
        k = len(words)
        bigrams = ngrams(words,2)
        trigrams = ngrams(words,3)
        if len(words) == 1:
          bigrams,trigrams = [(words)],[(words)]
        matches_for_a_keyphrase = []
        for tup in bigrams:
          sbi = ' '.join(list(tup))
          for s,p,o in triple_list:
            embedding1 = model.encode(sbi, convert_to_tensor=True)
            embedding2 = model.encode(p+" "+o, convert_to_tensor=True)
            cosine_scores = util.pytorch_cos_sim(embedding1, embedding2)
            if cosine_scores.item() > 0.45:
              matches_for_a_keyphrase.append([cosine_scores.item(),(s,p,o),kp])
        for tup in trigrams:
          sbi = ' '.join(list(tup))
          for s,p,o in triple_list:
            embedding1 = model.encode(sbi, convert_to_tensor=True)
            embedding2 = model.encode(p+" "+o, convert_to_tensor=True)
            cosine_scores = util.pytorch_cos_sim(embedding1, embedding2)
            if cosine_scores.item() > 0.45:
              matches_for_a_keyphrase.append([cosine_scores.item(),(s,p,o),kp])
        
        matches_for_a_keyphrase.sort(reverse = True)
        triple_matches = []
        # Now, we have to keep only (s,p,o) from matches_for_a_keyphrase in triple_matches
        for _,triple,_ in matches_for_a_keyphrase:
          triple_matches.append(triple) 
        matches.extend(triple_matches[:k+1])
        
      matches.sort(reverse = True)
      matches_dict[sent] = list(set(matches))   
    return matches_dict

In [ ]:
politician_trip

In [ ]:
# matches_pol = {}
# for sent_list, triple_list, keyphrase_scores in tqdm(zip(politician_sent, politician_trip,politicians_keyphrase_scores)):
#     for k, v in matches_kp(sent_list, triple_list, keyphrase_scores).items():
#         matches_pol[k] = v


# pol_matches_sent = [list(matches_pol.keys())]
# pol_matches_trips = list(matches_pol.values())
# ner_matches_pol = {}
# for sent_list, triple_list in tqdm(zip(politician_sent, politician_trip)):
#     for k, v in matches(sent_list, triple_list).items():
#         ner_matches_pol[k] = v

matches_act = {}
for sent_list, triple_list, keyphrase_scores in tqdm(zip(actors_sent, actors_trip,actors_keyphrase_scores)):
    for k, v in matches_kp(sent_list, triple_list,keyphrase_scores).items():
        matches_act[k] = v

# matches_cric = {}
# for sent_list, triple_list, keyphrase_scores in tqdm(zip(cricketers_sent, cricketers_trip, cricketers_keyphrase_scores)):
#     for k, v in matches_kp(sent_list, triple_list,keyphrase_scores).items():
#         matches_cric[k] = v

# matches_pol = {}
# for sent_list, triple_list in tqdm(zip(pol_sent, pol_trip)):
#     for k, v in matches_kp(sent_list, triple_list).items():
#         matches_pol[k] = v

50it [01:12,  1.45s/it]


### Evaluation : Precision and Recall

In [ ]:
politicians_test_dict_corrected = json.load(open('/content/drive/MyDrive/test_data/corrected_annotated_data/politicians_test(corrected).json',encoding = 'utf-8'))
for sent in politicians_test_dict_corrected:
  triplist = politicians_test_dict_corrected[sent]
  tup = set()
  for s,p,o in triplist:
    tup.add((s,p,o))
  politicians_test_dict_corrected[sent] = tup

In [4]:
def evaluate(test_dict, matches_dict):
    sum_prec = 0
    for key, val in matches_dict.items():
        tp, fp = 0, 0
        for k, v in test_dict.items():
            # If sentence matches
            if k == key:
                for ent in v:
                    for trip in val:
                        if ent == trip:
                            tp = tp + 1
                for trip in val:
                    flag = 0
                    for ent in v:
                        if ent == trip:
                            flag = 1
                            break
                    if flag == 0:
                        fp = fp + 1
                break
        if (tp+fp) != 0:
            prec = tp/(tp + fp)
        else:
            prec = 0
        sum_prec = prec + sum_prec

    sum_rec = 0
    for k, v in test_dict.items():
        rec = 0
        tp, fp = 0, 0
        for key, val in matches_dict.items():
            # If sentence matches
            if k == key:
                for ent in v:
                    for trip in val:
                        if ent == trip:
                            tp = tp + 1
                for trip in val:
                    flag = 0
                    for ent in v:
                        if ent == trip:
                            flag = 1
                            break
                    if flag == 0:
                        fp = fp + 1
                break
        rec = tp/len(v)
        sum_rec = rec + sum_rec

    avg_rec, avg_prec = sum_rec/len(test_dict), sum_prec/len(matches_dict)
    return avg_rec, avg_prec

avg_rec_act, avg_prec_act = evaluate(actors_test, actors_matches)
# avg_rec_cric, avg_prec_cric = evaluate(cricketers_test_dict, matches_cric)
# avg_rec_pol, avg_prec_pol = evaluate(politicians_test_dict_corrected, matches_pol)

In [5]:
avg_rec_act, avg_prec_act

(0.8469387755102041, 0.6542824074074075)

In [19]:
politicians_test_dict_corrected = json.load(open('/content/drive/MyDrive/test_data/corrected_annotated_data/politicians_test(corrected).json',encoding = 'utf-8'))
# for sent in politicians_test_dict_corrected:
#   triplist = politicians_test_dict_corrected[sent]
#   tup = set()
#   for s,p,o in triplist:
#     tup.add((s,p,o))
#   politicians_test_dict_corrected[sent] = tup

actors_test = json.load(open('/content/drive/MyDrive/Transformer Output/actors_test.json',encoding = 'utf-8'))
cricketers_test = json.load(open('/content/drive/MyDrive/Transformer Output/cricketers_test.json',encoding = 'utf-8'))

actors_matches = json.load(open('/content/drive/MyDrive/Transformer Output/actors_matches.json',encoding = 'utf-8'))
cricketers_matches = json.load(open('/content/drive/MyDrive/Transformer Output/cricketers_matches.json',encoding = 'utf-8'))
politicians_matches = json.load(open('/content/drive/MyDrive/Transformer Output/politicians_matches.json',encoding = 'utf-8'))

In [20]:
def global_evaluate(test_dict, matches_dict):
    sum_tp,sum_fp = 0,0
    total_count = 0
    for key, val in matches_dict.items():
        tp, fp = 0, 0
        for k, v in test_dict.items():
            # If sentence matches
            if k == key:
                for ent in v:
                    for trip in val:
                        if ent == trip:
                            tp = tp + 1
                for trip in val:
                    flag = 0
                    for ent in v:
                        if ent == trip:
                            flag = 1
                            break
                    if flag == 0:
                        fp = fp + 1
                break
        sum_tp += tp
        sum_fp += fp

    
    for k, v in test_dict.items():
        total_count += len(v)
    
    prec = sum_tp/(sum_tp+sum_fp)
    rec = sum_tp/(total_count)
    return rec,prec
avg_rec_act, avg_prec_act = global_evaluate(actors_test, actors_matches)
avg_rec_cric, avg_prec_cric = global_evaluate(cricketers_test, cricketers_matches)
avg_rec_pol, avg_prec_pol = global_evaluate(politicians_test_dict_corrected, politicians_matches)

In [ ]:
politicians_test_dict_corrected

In [ ]:
politicians_keyphrase_scores

In [ ]:
count  = 0
for sent in politicians_test_dict_corrected:
  if sent in matches_pol:
    triples_test = politicians_test_dict[sent]
    triples_matches = matches_pol[sent]
    for t in triples_test:
      if t not in triples_matches:
          print(t)
          count +=1
print(count)


  # else:
  #   print(politicians_test_dict[sent])


In [ ]:
for sent in politicians_test_dict_corrected:
  if sent in matches_pol:
    triples_test = politicians_test_dict[sent]
    triples_matches = matches_pol[sent]
    for t in triples_matches:
      if t not in triples_test:
        print(t)

In [ ]:
(avg_rec_act,avg_prec_act)

(0.7057823129251702, 0.6612857142857144)

In [22]:
(avg_rec_act,avg_prec_act), (avg_rec_cric, avg_prec_cric) , (avg_rec_pol, avg_prec_pol)

((0.8793103448275862, 0.6538461538461539),
 (0.9305135951661632, 0.7064220183486238),
 (0.8015267175572519, 0.6158357771260997))

In [23]:
AverageRecall = (avg_rec_act + avg_rec_cric + avg_rec_pol)/3
AveragePrecision = (avg_prec_act + avg_prec_cric + avg_prec_pol)/3

AverageRecall, AveragePrecision

(0.870450219183667, 0.6587013164402924)

In [ ]:
politicians_test_dict

### Output

In [ ]:
len(matches_act),len(matches_cric),len(matches_pol)

So, 7 sentences didn't find any matches at all.Let's see which ones

In [ ]:
for sent in politicians_test_dict:
  if sent not in matches_pol:
    print(sent," : ", politicians_test_dict[sent] )

Saving as json

In [ ]:
# for sent in actors_test_dict:
#   actors_test_dict[sent] = list(actors_test_dict[sent])
# for sent in cricketers_test_dict:
#   cricketers_test_dict[sent] = list(cricketers_test_dict[sent])
# for sent in politicians_test_dict:
#   politicians_test_dict[sent] = list(politicians_test_dict[sent])

# for sent in matches_act:
#   matches_act[sent] = list(matches_act[sent])
# for sent in matches_cric:
#   matches_cric[sent] = list(matches_cric[sent])
for sent in matches_pol:
  matches_pol[sent] = list(matches_pol[sent])

In [ ]:
# # saving the input test files
# with open("/content//drive//MyDrive//Transformer Output//actors_test.json", "w") as outfile: 
#     json.dump(actors_test_dict,outfile)
# with open("/content/drive/MyDrive/Transformer Output/cricketers_test.json", "w") as outfile: 
#     json.dump(cricketers_test_dict, outfile)
# with open("/content/drive/MyDrive/Transformer Output/politicians_test.json", "w") as outfile: 
#     json.dump(politicians_test_dict, outfile)

# #saving the output files
# with open("/content/drive/MyDrive/Transformer Output/actors_matches.json", "w") as outfile: 
#     json.dump(matches_act, outfile)
# with open("/content/drive/MyDrive/Transformer Output/cricketers_matches.json", "w") as outfile: 
#     json.dump(matches_cric, outfile)
with open("/content/drive/MyDrive/Transformer Output/politicians_matches_ner.json", "w") as outfile: 
    json.dump(matches_pol, outfile)

#### We do find quite a few of the matching triples to be relevant to the sentence. But, there are a few irrelevant matches as well.
Upon analysis, we think the word overlap is working better than the vector similarity approach. A possible reason can be that when we simply average out the words in a sentence, and when we average out the words in the triples and then find the similarity between these two averages, some semantic information is lost. So, triples that should have been irrelevant are also found as similar. As the word overlap method is a strictly string overlap, the relevance is much stronger.